#Modelos ARIMA/SARIMA

Estos modelos combinan una parte Autoregresiva (AR) con una parte de promediado de medias (MA), junto con diferenciación cuando hace falta (I): AR+I+MA

Si eso tambien lo hacen teniendo en cuenta una parte estacional (S), se covierte en modelo S+AR+I+MA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

###Predicción con series univariantes

In [ ]:
airline = pd.read_csv("airline_passengers.csv")
airline.head()

In [ ]:
airline.info()

In [ ]:
# Visualizamos
from matplotlib import pyplot as plt
airline['Thousands of Passengers'].plot(kind='line', figsize=(8, 4), title='Thousands of Passengers')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
airline.Month = pd.to_datetime(airline["Month"])
airline.set_index("Month", inplace=True)
airline.head()

In [ ]:
# Aseguramos que tenemos frecuancia mensual (mejor dicho, la forzamos)
airline.index.freq = 'MS'
airline.head()

Comprobamos si se han generado NA's

In [ ]:
airline.isna().sum()

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(airline)
plt.xticks(rotation = 90)
plt.xlabel("Fecha")
plt.ylabel("Número de pasajeros")

División entrenamiento y test:

In [ ]:
size = int(len(airline)*0.8)

train = airline[:size]
test = airline[size:]

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(train)
plt.plot(test)
#plt.xticks(rotation = 90)
plt.xlabel("Fecha")
plt.ylabel("Número de pasajeros")

Vamos ahora a descomponer la serie en estacionalidad, tendencia y residuos:

In [ ]:
descomposicion_multiplicativa = seasonal_decompose(
    train["Thousands of Passengers"],
    model = "multiplicative")
descomposicion_multiplicativa.plot()
plt.show()

####Análisis de la tendencia

**Análisis de la estacionariedad PARA LA TENDENCIA:**

Aplicamos Dickey Fuller para determinar si la serie de tendencia es estacionaria

In [ ]:
sts.adfuller(descomposicion_multiplicativa.trend.dropna())

Como no lo es, vamos a diferenciar:

In [ ]:
sts.adfuller(descomposicion_multiplicativa.trend.dropna().diff()[1:])

Vemos que el p-valor devuelto por la serie temporales es mayor a 0.05, por lo que la serie no parece ser estacionaria.

Podemos diferenciar de nuevo y ver si ya se ha convertido en estacionaria:

In [ ]:
sts.adfuller(descomposicion_multiplicativa.trend.dropna().diff()[1:].diff()[1:])

La serie de tendencia, con **dos diferenciaciones** ya es estacionaria

Vamos ahora a determinar el valor de p y el valor de q a partir de la función de autocorrelación y autocorrelación parcial:

**Función de autocorrelación (cálculo de q)**

Esta es la parte MA de la tendencia

In [ ]:
sgt.plot_acf(descomposicion_multiplicativa.trend.dropna().diff()[1:].diff()[1:],
             zero = False)
plt.title("Autocorrelación (tendencia)", size = 12)
plt.show()

q = 1

**Función de autocorrelación parcial (cálculo de p) PACF**

Esta es la parte AR de la tendencia

In [ ]:
sgt.plot_pacf(descomposicion_multiplicativa.trend.dropna().diff()[1:].diff()[1:],
              zero = False, method = ('ols'))
plt.title("Autocorrelación parcial (tendencia)", size = 12)
plt.show()

p = 2

####Análisis de la estacionalidad

**Análisis de la estacionariedad PARA LA ESTACIONALIDAD:**

In [ ]:
sts.adfuller(descomposicion_multiplicativa.seasonal)

La serie de estacionalidad **no necesita de diferenciación para ser estacionaria.**

Vamos ahora a calcular los valores de P y de Q a través de la función de autocorrelación y la función de autocorrelación parcial:

**Función de autocorrelación (estimación de Q)**

Esta es la parte MA de la estacionalidad

In [ ]:
sgt.plot_acf(descomposicion_multiplicativa.seasonal)
plt.title("Autocorrelación (estacionalidad)", size = 12)
plt.show()

Q = 2

**Función de autocorrelación parcial (estimación de P)**

Esta es la parte AR de la estacionalidad

In [ ]:
sgt.plot_pacf(descomposicion_multiplicativa.seasonal,
              zero = False,
              method = ('ols'))
plt.title("Autocorrelación parcial (estacionalidad)", size = 12)
plt.show()

P = 2

#####Análisis de residuos para la serie temporal

**Análisis de los residuos de la serie temporal**

In [ ]:
sts.adfuller(descomposicion_multiplicativa.resid.dropna())

Visualizamos la distribución de los residuos:

In [ ]:
sns.histplot(descomposicion_multiplicativa.resid.dropna())

#####Ajuste del modelo SARIMAX

**Ajuste del modelo SARIMAX:**

In [ ]:
modeloSARIMAX = SARIMAX(train,
                        order=(2, 2, 1), # (p, d, q)
                        seasonal_order=(2, 0, 2, 12),  #(P,D,Q,m)
                        trend='ct',
                        enforce_stationarity=False, enforce_invertibility=False
                        )
resultados = modeloSARIMAX.fit(maxiter=200)

In [ ]:
inicio_prediccion = len(train)
fin_prediccion = len(test)
test["prediccion"] = resultados.predict(
    start=inicio_prediccion,end=inicio_prediccion + fin_prediccion)

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(train, label="Train") #azul
plt.plot(test["Thousands of Passengers"], label="Test") #naranja
plt.plot(test["prediccion"], label="Prediction") #verde
#plt.xticks(rotation = 90)
plt.legend()
plt.xlabel("Fecha")
plt.ylabel("Número de pasajeros")

#####**Cálculo del error**

Error mape:

In [ ]:
mape = ((test["Thousands of Passengers"] -
         test["prediccion"])/
        test["Thousands of Passengers"]).abs() * 100
pd.DataFrame(mape).head(15)

In [ ]:
pd.DataFrame(mape).tail(15)

Error wape:

In [ ]:
wape = ((test["prediccion"] -
         test["Thousands of Passengers"]).abs().sum()/
        test["Thousands of Passengers"].abs().sum()) * 100
print(f"El error WAPE es de: {wape}%")

NOTA:

El Error Porcentual Absoluto Ponderado (*Weighted Absolute Percentage Error o  WAPE*) mide la desviación global de los valores pronosticados con respecto a los valores observados. El WAPE se calcula tomando la suma de los valores observados y la suma de los valores pronosticados, y calculando el error entre esos dos valores. Un valor más bajo indica un modelo más preciso.

##Cuestiones

- Prueba diferentes parámetros, intentando obtener el mejor resultado
- Hemos probado un modelo SARIMAX directamente. ¿Crees que un modelo ARIMA lo haría mejor?

In [ ]:
modeloARIMA = ARIMA(train, order=(2, 2, 1))
resultados = modeloARIMA.fit()

In [ ]:
inicio_prediccion = len(train)
fin_prediccion = len(test)
test["prediccion"] = resultados.predict(
    start=inicio_prediccion,end=inicio_prediccion + fin_prediccion)

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(train, label="Train") #azul
plt.plot(test["Thousands of Passengers"], label="Test") #naranja
plt.plot(test["prediccion"], label="Prediction") #verde
#plt.xticks(rotation = 90)
plt.legend()
plt.xlabel("Fecha")
plt.ylabel("Número de pasajeros")

In [ ]:
wape = ((test["prediccion"] -
         test["Thousands of Passengers"]).abs().sum()/
        test["Thousands of Passengers"].abs().sum()) * 100
print(f"El error WAPE es de: {wape}%")

# EOF